In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from cot.config import SAVE_DIR

In [3]:
exp = 1
attention_eval = True

save_dir = SAVE_DIR / f"res-cot-exp{exp}"
save_dir.mkdir(parents=True, exist_ok=True)

In [4]:
x = np.arange(4, 32)
y = np.arange(8, 128)
z = np.arange(0, 5001, 10)

In [5]:
Z1_parity = np.load(save_dir / "train_acc_parity.npy")
Z2_parity = np.load(save_dir / "test_acc_parity.npy")
Z1_nocot = np.load(save_dir / "train_acc_nocot.npy")
Z2_nocot = np.load(save_dir / "test_acc_nocot.npy")
Z1_copy = np.load(save_dir / "train_acc_copy.npy")
Z2_copy = np.load(save_dir / "test_acc_copy.npy")

if attention_eval:
    Z3_parity = np.load(save_dir / "attn0_parity.npy")
    Z4_parity = np.load(save_dir / "attn1_parity.npy")
    Z3_nocot = np.load(save_dir / "attn0_nocot.npy")
    Z4_nocot = np.load(save_dir / "attn1_nocot.npy")
    Z3_copy = np.load(save_dir / "attn0_copy.npy")
    Z4_copy = np.load(save_dir / "attn1_copy.npy")

In [6]:
for Zi in [
    Z1_parity,
    Z2_parity,
    Z3_parity,
    Z4_parity,
    Z1_nocot,
    Z2_nocot,
    Z3_nocot,
    Z4_nocot,
    Z1_copy,
    Z2_copy,
    Z3_copy,
    Z4_copy,
]:
    tmp = np.repeat(Zi.mean(axis=1)[:, np.newaxis, :], Zi.shape[1], axis=1)
    Zi[Zi == -1] = tmp[Zi == -1]

In [7]:
X, Y, Z = np.meshgrid(y, x, z)

In [8]:
data = Z1_parity
kw = {
    "vmin": 0,
    "vmax": 1,
    "levels": 100,
    "cmap": "RdBu_r",
    "rasterized": True,
}


In [9]:
# Create a figure with 3D ax
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(111, projection="3d")

ax.set_xlabel(r"Bandwidth $\sigma$", fontsize=10)
ax.set_ylabel(r"Regularizer $\lambda$", fontsize=10)
ax.set_zlabel(r"Samples $n$", fontsize=10)

# Plot contour
xmin, xmax = X.min(), X.max()
ymin, ymax = Y.min(), Y.max()
zmin, zmax = Z.min(), Z.max()
ax.set(xlim=[xmin, xmax], ylim=[ymax, ymin], zlim=[zmin, zmax])
edges_kw = dict(color="0.4", linewidth=1, zorder=1e3, linestyle="--")
ax.plot([xmax, xmax], [ymax, ymin], zmax, **edges_kw)
ax.plot([xmin, xmax], [ymin, ymin], zmax, **edges_kw)
ax.plot([xmax, xmax], [ymin, ymin], [zmin, zmax], **edges_kw)

# Cube surfaces
C = ax.contourf(
    X[:, :, -1], Y[:, :, -1], data[:, :, -1], zdir="z", offset=Z.max(), **kw
)
for c in C.collections:
    c.set_rasterized(True)
C = ax.contourf(X[0, :, :], data[0, :, :], Z[0, :, :], zdir="y", offset=Y.min(), **kw)
for c in C.collections:
    c.set_rasterized(True)
Cs = ax.contourf(data[:, -1, :], Y[:, -1, :], Z[:, -1, :], zdir="x", offset=X.max(), **kw)
for c in Cs.collections:
    c.set_rasterized(True)

n1, n2 = 3, 30
n1, n2 = 30, 300
ax.set_zticks([np.log10(n1), np.log10(n2)])
ax.set_zticklabels([rf"${n1}$", rf"${n2}$"])

s1, s2, s3 = .8, .5, .2
ax.set_xticks([np.log10(s1), np.log10(s2), np.log10(s3)])
ax.set_xticklabels([rf"${s1}$", rf"${s2}$", rf"${s3}$"])

l1, l2, l3 = -1, -2, -3
ax.set_yticks([l1, l2, l3])
ax.set_yticklabels(
    [r"$10^{" + str(l1) + "}$", r"$10^{" + str(l2) + "}$", r"$10^{" + str(l3) + "}$"]
)

ax.view_init(elev=30, azim=40)  # , vertical_axis='x')
# fig.savefig(SAVE_DIR / "block_edge.pdf", bbox_inches='tight')
fig.savefig(SAVE_DIR / "block_edge_zoom.pdf", dpi=300, bbox_inches="tight")

fig, ax = plt.subplots(figsize=(1, 3))
ax.axis("off")
c = fig.colorbar(C, ax=ax, ticks=[-4, -3])
c.ax.set_yticklabels([r"$10^{-4}$", r"$10^{-3}$"])
fig.savefig(SAVE_DIR / "block_step_colorbar.pdf", bbox_inches="tight") 

/tmp/ipykernel_3961714/1160597982.py:20: UserWarning: The following kwargs were not used by contour: 'rasterized'
  C = ax.contourf(
/tmp/ipykernel_3961714/1160597982.py:23: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  for c in C.collections:
/tmp/ipykernel_3961714/1160597982.py:25: UserWarning: The following kwargs were not used by contour: 'rasterized'
  C = ax.contourf(X[0, :, :], data[0, :, :], Z[0, :, :], zdir="y", offset=Y.min(), **kw)
/tmp/ipykernel_3961714/1160597982.py:26: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  for c in C.collections:


/tmp/ipykernel_3961714/1160597982.py:28: UserWarning: The following kwargs were not used by contour: 'rasterized'
  Cs = ax.contourf(data[:, -1, :], Y[:, -1, :], Z[:, -1, :], zdir="x", offset=X.max(), **kw)
/tmp/ipykernel_3961714/1160597982.py:29: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  for c in Cs.collections:


AttributeError: 'PathCollection' object has no attribute 'do_3d_projection'

Error in callback <function _draw_all_if_interactive at 0x7efd2a3e1580> (for post_execute), with arguments args (),kwargs {}:


AttributeError: 'PathCollection' object has no attribute 'do_3d_projection'

AttributeError: 'PathCollection' object has no attribute 'do_3d_projection'

<Figure size 500x400 with 1 Axes>